In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-0a9c6883-18dc-62c0-a8c5-040b2b9e98a6)


In [3]:
import pandas as pd
df = pd.read_csv('/kaggle/input/iam-text/words_new.txt', header = None, sep = " ", on_bad_lines= 'warn')
df.rename(columns={0: "file_name", 8: "text"}, inplace=True)
del df[1], df[2], df[3], df[4], df[5], df[6], df[7]
df = df[df['text'] != 'at']
df = df[df['text'] != ',']
df = df[df['file_name'] != 'a01-117-05-02']
df = df[df['text'] != ')']
#df = df[df['text'] != ' ']
df.reset_index(drop=True, inplace=True)
df

Skipping line 48: expected 9 fields, saw 10
Skipping line 100: expected 9 fields, saw 10
Skipping line 19778: expected 9 fields, saw 11
Skipping line 19960: expected 9 fields, saw 11
Skipping line 20711: expected 9 fields, saw 11
Skipping line 20743: expected 9 fields, saw 10
Skipping line 20784: expected 9 fields, saw 11
Skipping line 20801: expected 9 fields, saw 11
Skipping line 20822: expected 9 fields, saw 11
Skipping line 20853: expected 9 fields, saw 11
Skipping line 20898: expected 9 fields, saw 11
Skipping line 20930: expected 9 fields, saw 10
Skipping line 20946: expected 9 fields, saw 11
Skipping line 20951: expected 9 fields, saw 10
Skipping line 21001: expected 9 fields, saw 10
Skipping line 21085: expected 9 fields, saw 10
Skipping line 21102: expected 9 fields, saw 11
Skipping line 21162: expected 9 fields, saw 11
Skipping line 21197: expected 9 fields, saw 10



,file_name,text
0,a01-000u-00-00,A
1,a01-000u-00-01,MOVE
2,a01-000u-00-02,to
3,a01-000u-00-03,stop
4,a01-000u-00-04,Mr.
...,...,...
30163,e07-072-08-06,can
30164,e07-072-08-07,be
30165,e07-072-09-00,obtained
30166,e07-072-09-02,a


In [4]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # some file names end with jp instead of jpg, the two lines below fix this
        part = file_name.split('-')
        file_name = file_name + '.png'
        file_path = self.root_dir+ '/' + part[0] + '/' + part[0] + '-' + part[1] + '/' + file_name
        # prepare image (i.e. resize + normalize)
        image = Image.open(file_path).convert("RGB")

        try: 
            pixel_values = self.processor(image, return_tensors="pt").pixel_values
        except:
            pixel_values = torch.zeros((3, 384, 384))
            #lablels = torch.zeros((128))
            #encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
            #return encoding
            #pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          truncation=True,
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        labels = labels + [-100] * (self.max_target_length - len(labels))

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [5]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [6]:
dataset = IAMDataset(root_dir = '/kaggle/input/iam-handwriting-word-database/iam_words/words', df = df, processor = processor)
len(dataset)

30168

In [7]:
train_size = 0.8
valid_size = 0.2

In [8]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset,
                                    [int(train_size * len(dataset)) + 1,
                                     int(valid_size * len(dataset))])
train_set[0]['labels'].shape

torch.Size([128])

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(valid_set, batch_size=4)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [11]:
encoding = dataset[1]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


Importantly, we need to set a couple of attributes, namely:

the attributes required for creating the decoder_input_ids from the labels (the model will automatically create the decoder_input_ids by shifting the labels one position to the right and prepending the decoder_start_token_id, as well as replacing ids which are -100 by the pad_token_id)
the vocabulary size of the model (for the language modeling head on top of the decoder)
beam-search related parameters which are used when generating text.

In [12]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4


In [13]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
!pip install -q datasets jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [16]:
def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

In [17]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
from transformers import AdamW
from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
import wandb


In [20]:
import time

def train(train_dataloader, valid_dataloader, epoch, display_step):
    print(f"Start epoch #{epoch+1}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    train_loss = 0.0

    for i, batch in enumerate(tqdm(train_dataloader)):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)

      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      #train_loss += loss.item()

      #print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

      # Apply gradients
      optimizer.step()

      # Save loss
      train_loss_epoch += loss.item()
      if (i) % display_step == 0:
        print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f}'.format(
                epoch + 1, (i+1) * len(batch[k]), len(train_dataloader.dataset), 100 * (i+1) * len(batch[k]) / len(train_dataloader.dataset),
                loss.item()))

    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch/= (i + 1)

    # Evaluate the validation set
    model.eval()
    valid_cer = 0.0
    with torch.no_grad():
      for batch in tqdm(eval_dataloader):
        # run batch generation
        outputs = model.generate(batch["pixel_values"].to(device))
        # compute metrics
        cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
        valid_cer += cer

    print("Validation CER:", valid_cer / len(eval_dataloader))

    test_loss_epoch/= (i+1)

    return train_loss_epoch , test_loss_epoch

In [21]:
wandb.login(
    key = "82599b022f523294b4afeedd55c8e4fcf3c86a48",
)
wandb.init(
    project = "trOCR"
)
epochs = 7
display_step = 1000

# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0

    (train_loss_epoch, test_loss_epoch) = train(train_dataloader,
                                              eval_dataloader, epoch, display_step)

    if test_loss_epoch < last_loss:
        last_loss = test_loss_epoch

    train_loss_array.append(train_loss_epoch)
    test_loss_array.append(test_loss_epoch)
    wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch})

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vutuanminh-work (deeplearningproj). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231215_160109-eb36hnw5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run twilight-moon-32
wandb: ⭐️ View project at https://wandb.ai/deeplearningproj/trOCR
wandb: 🚀 View run at https://wandb.ai/deeplearningproj/trOCR/runs/eb36hnw5


Start epoch #1


  0%|          | 0/6034 [00:00<?, ?it/s]

Train Epoch: 1 [4/24135 (0.016573441060700227%)]	Loss: 10.7404
Train Epoch: 1 [4004/24135 (16.59001450176093%)]	Loss: 0.9667
Train Epoch: 1 [8004/24135 (33.16345556246116%)]	Loss: 1.2340
Train Epoch: 1 [12004/24135 (49.736896623161385%)]	Loss: 1.6442
Train Epoch: 1 [16004/24135 (66.31033768386162%)]	Loss: 0.8416
Train Epoch: 1 [20004/24135 (82.88377874456184%)]	Loss: 0.5290
Train Epoch: 1 [24004/24135 (99.45721980526207%)]	Loss: 0.9964
Done epoch #1, time for this epoch: 3762.3941292762756s


  0%|          | 0/1509 [00:00<?, ?it/s]

...

Validation CER: 0.15379632458518108
Start epoch #7


  0%|          | 0/6034 [00:00<?, ?it/s]

Train Epoch: 7 [4/24135 (0.016573441060700227%)]	Loss: 0.4712
Train Epoch: 7 [4004/24135 (16.59001450176093%)]	Loss: 0.6771
Train Epoch: 7 [8004/24135 (33.16345556246116%)]	Loss: 0.5631
Train Epoch: 7 [12004/24135 (49.736896623161385%)]	Loss: 0.5199
Train Epoch: 7 [16004/24135 (66.31033768386162%)]	Loss: 0.4090
Train Epoch: 7 [20004/24135 (82.88377874456184%)]	Loss: 0.4626
Train Epoch: 7 [24004/24135 (99.45721980526207%)]	Loss: 0.5590
Done epoch #7, time for this epoch: 3419.705637693405s


  0%|          | 0/1509 [00:00<?, ?it/s]

Validation CER: 0.16132414534237988


In [22]:
import torch 
torch.save(model.state_dict(),'/kaggle/working/trOCR.pth')